# Chatbots

In this notebook, you will learn how to store conversation history and thereby enable chatbot functionality in your LLM-based chains.

---

## Objectives

By the time you complete this notebook you will:

- Understand the core principles and techniques required to create chatbot applications capable of retaining conversation history.
- Create easy-to-use chatbots, capable of assuming a variety of different personas.
- Interact with a simple chatbot application interface, well-suited for chatbot application prototyping.

---

## Imports

In [1]:
!pip install groq langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 5.2 MB/s eta 0:00:00


In [2]:
import os
import getpass

os.environ["GROQ_API_KEY"] = getpass.getpass("GROQ API Key:\n")

GROQ API Key:
··········


In [3]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

---

## Create a Model Instance

In [4]:
llm = ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0)

---

## Placeholder Messages

Before we begin work enabling conversation history and chatbot functionality, we need to introduce a new kind of message that we have not yet covered, **placeholder** messages.

Put simply, and as the name suggest, a placeholder message is used in a prompt template to hold the place of a list of other messages.

In [5]:
template_with_placeholder = ChatPromptTemplate.from_messages([
    ('placeholder', '{messages}'),
    ('human', '{prompt}')
])

In [6]:
messages = [
    ('human', 'The sun came up today.'),
    ('ai', 'That is wonderful!'),
    ('human', 'The sun went down today.'),
    ('ai', 'That is also wonderful!.')
]

In [7]:
prompt = 'What happened today?'

When invoking (or streaming or batching) prompt templates or chains that contain placeholder messages, we provide a value as the template indicates, only in the case of a placeholder message, we provide a list of other messages, rather than a string.

Here we invoke `template_with_placeholder`, passing in the `messages` list to fulfil the template's `messages` parameter, and the `prompt` string to fulfil its `prompt` parameter.

In [8]:
template_with_placeholder.invoke({'messages': messages, 'prompt': prompt})

ChatPromptValue(messages=[HumanMessage(content='The sun came up today.', additional_kwargs={}, response_metadata={}), AIMessage(content='That is wonderful!', additional_kwargs={}, response_metadata={}), HumanMessage(content='The sun went down today.', additional_kwargs={}, response_metadata={}), AIMessage(content='That is also wonderful!.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What happened today?', additional_kwargs={}, response_metadata={})])

As you can see, LangChain expanded the `placeholder` message, for which we provided a list, into a list of individual messages that we provided when invoking the prompt template.

It should come as no surprise that we can use this prompt template in chains just as we would any other.

In [9]:
chain = template_with_placeholder | llm | StrOutputParser()

In [10]:
chain.invoke({'messages': messages, 'prompt': prompt})

"According to our conversation, the sun came up and then went down today, which is a normal part of a day. Is there anything else you'd like to share or talk about that happened today?"

---

## Rudimentary Conversation History

We can easily construct a rudimentary conversation history mechanism using a message placeholder. First we'll create a prompt template utilizing a placeholder, and use it in a simple chain.

In [11]:
chat_conversation_template = ChatPromptTemplate.from_messages([
    ('placeholder', '{chat_conversation}')
])

In [12]:
chat_chain = chat_conversation_template | llm | StrOutputParser()

Next we'll create a list to store our conversation, which we will add to over time.

In [13]:
chat_conversation = []

We will begin by appending our first prompt, as a `user` message to the `chat_conversation` list.

In [14]:
chat_conversation.append(('user', 'Hello, my name is Michael.'))

Just to test it out, we can now invoke our `chat_chain` with the current `chat_conversation` list.

In [15]:
chat_chain.invoke({'chat_conversation': chat_conversation})

"Hello Michael, it's nice to meet you. Is there something I can help you with or would you like to chat?"

It looks like the LLM is able to respond just fine. Since we are wanting to keep track of the conversation history, however, let's invoke the chain again, but this time append the response to the `chat_conversation` list as an `ai` message.

In [16]:
response = chat_chain.invoke({'chat_conversation': chat_conversation})
chat_conversation.append(('ai', response))

Looking at `chat_conversation` we see it now contains a list of the messages thus far.

In [17]:
chat_conversation

[('user', 'Hello, my name is Michael.'),
 ('ai',
  "Hello Michael, it's nice to meet you. Is there something I can help you with or would you like to chat?")]

Let's repeat this same process with a new message, and let's pass in a prompt that relies on previous conversation history to answer correctly.

In [18]:
chat_conversation.append(('user', 'Do you remember what my name is?'))

In [19]:
response = chat_chain.invoke({'chat_conversation': chat_conversation})
chat_conversation.append(('ai', response))
chat_conversation

[('user', 'Hello, my name is Michael.'),
 ('ai',
  "Hello Michael, it's nice to meet you. Is there something I can help you with or would you like to chat?"),
 ('user', 'Do you remember what my name is?'),
 ('ai',
  'Your name is Michael. I can remember that for the duration of our conversation.')]

As you can see, by appending user prompt and AI responses to `chat_conversation` as `user` and `ai` messages respectively, and then invoking our placeholder-containing `chat_chain` with the entire up-to-date conversation, we now have the ability to converse with the LLM in a way where it retains details from earlier in the conversation.

At its most basic level, all chatbot functionality that is capable of retaining conversation history utilizes this method of passing in the conversation prior to new user messages.

---

## Chatbot Class

We can encapsulate the functionality we acheived above into a class that will make interacting with our conversation history-enabled LLM much simpler. Please read following `Chatbot` class definition, including its comments, carefully.

In [20]:
class Chatbot:
    def __init__(self, llm):
        # This is the same prompt template we used earlier, which a placeholder message for storing conversation history.
        chat_conversation_template = ChatPromptTemplate.from_messages([
            ('placeholder', '{chat_conversation}')
        ])

        # This is the same chain we created above, added to `self` for use by the `chat` method below.
        self.chat_chain = chat_conversation_template | llm | StrOutputParser()

        # Here we instantiate an empty list that will be added to over time.
        self.chat_conversation = []

    # `chat` expects a simple string prompt.
    def chat(self, prompt):
        # Append the prompt as a user message to chat conversation.
        self.chat_conversation.append(('user', prompt))

        response = self.chat_chain.invoke({'chat_conversation': self.chat_conversation})
        # Append the chain response as an `ai` message to chat conversation.
        self.chat_conversation.append(('ai', response))
        # Return the chain response to the user for viewing.
        return response

    # Clear conversation history.
    def clear(self):
        self.chat_conversation = []

Let's instantiate a chatbot instance.

In [22]:
chatbot = Chatbot(llm)

We can now utilize its `chat` method.

In [23]:
print(chatbot.chat('Hi, my name is Michael.'))

Hello Michael, it's nice to meet you. Is there something I can help you with or would you like to chat?


In [24]:
print(chatbot.chat('I just want to be reminded of my name please.'))

Your name is Michael.


In [25]:
print(chatbot.chat("Tell me something interesting I probably don't know about pi."))

Here's something interesting about pi: Pi (π) is an irrational number, which means it cannot be expressed as a finite decimal or fraction. But what's even more fascinating is that pi is a "transcendental number", which means it's not just irrational, but it's also not a root of any polynomial equation with rational coefficients.

In simpler terms, this means that pi is not just a weird, never-ending decimal, but it's also fundamentally connected to the nature of mathematics itself. It's a number that shows up in many unexpected places, from geometry to calculus to number theory, and its unique properties have made it a subject of fascination for mathematicians and scientists for centuries.

Oh, and by the way, your name is still Michael.


In [26]:
print(chatbot.chat("That's really cool! Give me another.")) # Note we are not being specific about what "another" refers to...the LLM needs to have previous messages to understand our intent.

Here's another interesting fact about pi: Pi appears in the distribution of prime numbers. The prime number theorem, which describes how prime numbers are distributed among the integers, involves pi in a surprising way. Specifically, the theorem states that the number of prime numbers less than or equal to x grows like x / ln(x) as x approaches infinity, where ln(x) is the natural logarithm of x.

But here's the amazing part: the error term in this approximation, which describes how far off the approximation is from the true value, involves pi. In fact, the error term is proportional to x / (ln(x) ^ 2) * sqrt(2 * pi * x / ln(x)), which means that pi shows up in the description of the distribution of prime numbers.

This is a remarkable example of how pi, which is often thought of as a purely geometric constant, appears in unexpected places in mathematics, including number theory.

And, just to remind you, your name is still Michael.


---

---

## Exercise: Enable Role-based Chatbots

For this exercise you'll enable your chatbot instances to assume a specific role by leveraging a system message.

Below is the class definition for `ChatbotWithRole` which currently is identical to the `Chatbot` class definition above except that we've defined a `system_message` argument (defaulting to an empty string) that be used when instantiating `ChatbotWithRole` instances.

Edit the class definition as needed to that you can supply a system message that will create a specific role for your chatbot. Upon completion you should be able to use system messages like the following to create an overarching role for your chatbot to assume.

Feel free to check out the *Solution* below if you get stuck.

In [30]:
brief_chatbot_system_message = "You always answer as briefly and concisely as possible."

curious_chatbot_system_message = """\
You are incredibly curious, and often respond with reflections and followup questions that lean the conversation in the direction of playfully \
understanding more about the subject matters of the conversation."""

increased_vocabulary_system_message = """\
You always respond using challenging and often under-utilized vocabulary words, even when your response could be made more simply."""

### Your Work Here

Update the following class definition so that the passed-in `system_message` is effectively utilized by chatbot instances.

In [27]:
class ChatbotWithRole:
    def __init__(self, llm, system_message=''):
        # This is the same prompt template we used earlier, which a placeholder message for storing conversation history.
        chat_conversation_template = ChatPromptTemplate.from_messages([
            ('placeholder', '{chat_conversation}')
        ])

        # This is the same chain we created above, added to `self` for use by the `chat` method below.
        self.chat_chain = chat_conversation_template | llm | StrOutputParser()

        # Here we instantiate an empty list that will be added to over time.
        self.chat_conversation = []

    # `chat` expects a simple string prompt.
    def chat(self, prompt):
        # Append the prompt as a user message to chat conversation.
        self.chat_conversation.append(('user', prompt))

        response = self.chat_chain.invoke({'chat_conversation': self.chat_conversation})
        # Append the chain response as an `ai` message to chat conversation.
        self.chat_conversation.append(('ai', response))
        # Return the chain response to the user for viewing.
        return response

    # Clear conversation history.
    def clear(self):
        self.chat_conversation = []

### Try Out a Chatbot With a Role

After successfully implementing `ChatbotWithRole`, try creating an instance of it with a system message of your choosing and interact with it.

### Solution

The solution is brief. Here we added an additional system message to the `chat_conversation_template` that uses the passed-in `system_message`.

In [28]:
class ChatbotWithRole:
    def __init__(self, llm, system_message=''):
        # This is the same prompt template we used earlier, which a placeholder message for storing conversation history.
        chat_conversation_template = ChatPromptTemplate.from_messages([
            ('system', system_message),
            ('placeholder', '{chat_conversation}')
        ])

        # This is the same chain we created above, added to `self` for use by the `chat` method below.
        self.chat_chain = chat_conversation_template | llm | StrOutputParser()

        # Here we instantiate an empty list that will be added to over time.
        self.chat_conversation = []

    # `chat` expects a simple string prompt.
    def chat(self, prompt):
        # Append the prompt as a user message to chat conversation.
        self.chat_conversation.append(('user', prompt))

        response = self.chat_chain.invoke({'chat_conversation': self.chat_conversation})
        # Append the chain response as an `ai` message to chat conversation.
        self.chat_conversation.append(('ai', response))
        # Return the chain response to the user for viewing.
        return response

    # Clear conversation history.
    def clear(self):
        self.chat_conversation = []

Let's try it out with one of the system messages defined above.

In [31]:
brief_chatbot = ChatbotWithRole(llm, system_message=brief_chatbot_system_message)
curious_chatbot = ChatbotWithRole(llm, system_message=curious_chatbot_system_message)
increased_vocabulary_chatbot = ChatbotWithRole(llm, system_message=increased_vocabulary_system_message)

In [32]:
print(brief_chatbot.chat("What would you consider a good morning routine?"))

Wake up early, exercise, meditate, and eat a healthy breakfast.


In [33]:
print(curious_chatbot.chat("What would you consider a good morning routine?"))

A good morning routine can set the tone for the entire day, can't it? I think it's fascinating how different people have different approaches to starting their day. Some people swear by a rigorous exercise routine, while others prefer a more relaxed, meditative start.

For me, a good morning routine would involve a balance of physical and mental stimulation. Perhaps starting with some gentle stretching or yoga to get the blood flowing, followed by a short meditation or deep breathing exercise to clear the mind. And of course, a good cup of coffee or tea to provide a boost of energy!

But what I find really interesting is how morning routines can vary depending on individual personalities and lifestyles. For example, some people might prefer a more creative start to their day, like writing or drawing, while others might prioritize getting a head start on their work or responding to urgent emails.

What about you, what does your ideal morning routine look like? Do you have any favorite a

In [34]:
print(increased_vocabulary_chatbot.chat("What would you consider a good morning routine?"))

Initiating a diel rhythm that fosters an efficacious and salubrious morning routine is paramount. I would recommend commencing the day with a period of crepuscular contemplation, wherein one engages in a bout of introspective rumination, perhaps accompanied by a cup of caffeinated beverage to stimulate the cerebral cortex and facilitate a state of heightened vigilance.

Subsequently, a regimen of calisthenics or other forms of physical exertion, such as yoga or jogging, can serve to galvanize the muscles and invigorate the circulatory system, thereby enhancing one's overall sense of dynamism and élan vital. A nutritious and balanced breakfast, replete with an array of wholesome comestibles, would then provide the necessary sustenance to sustain one's energies throughout the morning.

Furthermore, incorporating a modicum of mental stimulation, such as perusing a literary work or engaging in a puzzle or brain teaser, can help to hone one's cognitive faculties and foster a sense of intell

---